In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay
from sklearn import metrics
import torchvision
from torchvision import datasets, models, transforms
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
import time
import os
from PIL import Image
from tempfile import TemporaryDirectory
from pathlib import Path
from pylab import savefig

cudnn.benchmark = True
plt.ion()   # interactive mode


In [2]:
# Data augmentation and normalization for training
# Just normalization for validation
def prepare_data(path):
    data_transforms = {
        'train': transforms.Compose([
        #transforms.RandomResizedCrop(224),
        #transforms.RandomHorizontalFlip(),
            transforms.Resize([224,224]),
          #  transforms.Resize([480,480]),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'test': transforms.Compose([
            transforms.Resize([224,224]),
        #transforms.CenterCrop(224),
           # transforms.Resize([480,480]),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }

    data_dir = path
    image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                              data_transforms[x])
                      for x in ['train', 'test']}
    dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=64,
                                                 shuffle=True, num_workers=4)
                  for x in ['train', 'test']}
    dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}
    class_names = image_datasets['train'].classes
    #print(class_names)
    #device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    return dataloaders, dataset_sizes

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs, fold_num):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("train model girildi")
    since = time.time()

    # Create a temporary directory to save training checkpoints
    with TemporaryDirectory() as tempdir:
        best_model_params_path = os.path.join(tempdir, 'best_model_params.pt')

        torch.save(model.state_dict(), best_model_params_path)
        best_acc = 0.0

        for epoch in range(num_epochs):
            print(f'Epoch {epoch}/{num_epochs - 1}')
            print('-' * 10)

            # Each epoch has a training and validation phase
            for phase in ['train', 'test']:
                if phase == 'train':
                    model.train()  # Set model to training mode
                else:
                    model.eval()   # Set model to evaluate mode

                running_loss = 0.0
                running_corrects = 0

                # Iterate over data.
                for inputs, labels in dataloaders[phase]:
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                if phase == 'train':
                    scheduler.step()

                epoch_loss = running_loss / dataset_sizes[phase]
                epoch_acc = running_corrects.double() / dataset_sizes[phase]

                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

                # deep copy the model
                if phase == 'test' and epoch_acc > best_acc:
                    best_acc = epoch_acc
                    torch.save(model.state_dict(), best_model_params_path)

            print()

        time_elapsed = time.time() - since
        print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
        print(f'Best val Acc: {best_acc:4f}')

        # load best model weights
        model.load_state_dict(torch.load(best_model_params_path))
        
    
    print("train model çıkıldı")
  
    return model

In [4]:
def visualize_model(model, num_images=4):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['test']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//3, 3, images_so_far)
                ax.axis('off')
                ax.set_title(f'predicted: {class_names[preds[j]]}')
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [ ]:
def conf_mat(model,fold_idx):
    class_names = ["Normal", "Tapered","Pyriform", "Amorphous"]
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("confmat girildi")
    fold_loss_values = []
    fold_correct = 0
    fold_total = 0
    fold_true = []
    fold_predicted = []
    model.eval()
    with torch.no_grad():
        for inputs, labels in dataloaders['test']:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            fold_total += labels.size(0)
            fold_correct += (predicted == labels).sum().item()
            fold_true.extend(labels.cpu().numpy())
            fold_predicted.extend(predicted.cpu().numpy())
                
    return(fold_total,fold_correct,fold_true,fold_predicted)

In [ ]:
def restart_model(opti,learning_rate_selection):
    print("restart model was started")
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    #print(device)
    #model_ft = models.efficientnet_v2_m(weights='IMAGENET1K_V1')
    #model_ft.name = 'efficientnet_v2_m'
    #model_ft = models.efficientnet_v2_s(weights='IMAGENET1K_V1')
    #model_ft.name = 'efficientnet_v2_s'
    model_ft = models.densenet201(weights='IMAGENET1K_V1')
    model_ft.name = 'Densenet201'
    #print(model_ft)
    #num_ftrs = model_ft.AuxLogits.fc.in_features # inception
    #model_ft.aux_logits=False
    
    #model_ft.AuxLogits.fc = nn.Linear(num_ftrs, 18) #inception
    
    #num_ftrs = model_ft.fc.in_features #ResNet
    num_ftrs = model_ft.classifier.in_features # DenseNet
    #num_ftrs = model_ft.classifier[1].in_features
    #num_ftrs = model_ft.heads[0].in_features
    #num_ftrs=model_ft.head.in_features        
        
    #model_ft.classifier[1] = nn.Linear(num_ftrs, 3)   #ConvNext için Dene
    #model_ft.head = nn.Linear(num_ftrs, 18) 

    model_ft.fc = nn.Linear(num_ftrs, 4)
    #print(model_ft)
    model_ft = model_ft.to(device)

    criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
    if opti== 'Adamax':
        optimizer_ft = optim.Adamax(model_ft.parameters(), lr=learning_rate_selection)
    elif opti == 'SGD':
        optimizer_ft = optim.SGD(model_ft.parameters(), lr=learning_rate_selection, momentum=0.9)
    elif opti == 'RMSprop':
        optimizer_ft = optim.RMSprop(model_ft.parameters(), lr=learning_rate_selection, momentum=0.9)


# Decay LR by a factor of 0.1 every 7 epochs
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=5, gamma=0.1)
    return(model_ft,criterion,optimizer_ft,exp_lr_scheduler)

In [ ]:
def genel_fold_basarisi(fold_total,fold_correct,fold_true,fold_predicted):
    print("Fold accuracy was written")
    fold_accuracy = (fold_correct / fold_total) * 100
    fold_accuracies.append(fold_accuracy)
    fold_true_labels.extend(fold_true)
    fold_predicted_labels.extend(fold_predicted)
    return(fold_accuracies,fold_true_labels,fold_predicted_labels)


In [ ]:
def fold_conf_mat(fold_true_labels,fold_predicted_labels,opti,learning_rate_selection,imageName,txtName,model_ft):
    print("fold conf mat girildi")
    class_names = ["Normal", "Tapered","Pyriform", "Amorphous"] 
    
    
    confusion_mtx = metrics.confusion_matrix(fold_true_labels, fold_predicted_labels)
    class_to_label = {'Normal': 0, 'Tapered': 1,'Pyriform':2, 'Amorphous': 3}

    
    df_cm = pd.DataFrame(confusion_mtx, index = [i for i in class_to_label],
                columns = [i for i in class_to_label])
    plt.figure(figsize = (10,10))
    plt.title(f'General Confusion Matrix for {model_ft.name} - {opti} - {learning_rate_selection}')
    result_confmat= sn.heatmap(df_cm, annot=True,cmap="OrRd",fmt="d",annot_kws={"size": 11},cbar=False)
    plt.ylabel('True labels',fontsize=12)
    plt.xlabel('Predicted labels',fontsize=12)
    figure = result_confmat.get_figure()    
    figure.savefig(imageName, dpi=400)
    g_fold_class_report = classification_report(fold_true_labels, fold_predicted_labels, target_names=class_names, digits=4)
    print(f"Classification Report for Dataset for {model_ft.name} - {opti} - {learning_rate_selection}:\n{g_fold_class_report}")
    f=open(txtName,"w")
    f.write(g_fold_class_report)
    f.close()

In [ ]:
dataset_path = '<Dataset Path>'
output_path = '<Output Path>'
num_epochs=20

In [ ]:
for opti in ['Adamax','RMSprop',  'SGD']:
    if opti== 'Adamax':
        Dizi = [0.001]
    elif opti == 'RMSprop':
        break
        Dizi = [0.0001]
    elif opti == 'SGD':
        break
        Dizi = [0.001]

    for learning_rate_selection in Dizi:
        fold_accuracies = []
        fold_true_labels = []
        fold_predicted_labels = []
        print(f'Optimizer: {opti}')
        print(f'Learning Rate: {learning_rate_selection}')

        dataset_dir = dataset_path + '/fold_1'
        dataloaders, dataset_sizes = prepare_data(dataset_dir)
        model_ft,criterion,optimizer_ft,exp_lr_scheduler=restart_model(opti,learning_rate_selection)

        imageName = output_path+'/'+ Path(dataset_path).parts[-1] +'_'+ model_ft.name + '_' + optimizer_ft.__class__.__name__ + '_' + str(learning_rate_selection) + '_Epoch_' + str(num_epochs) + '.png'
        txtName = output_path+'/'+ Path(dataset_path).parts[-1] +'_'+ model_ft.name + '_' + optimizer_ft.__class__.__name__ + '_' + str(learning_rate_selection) + '_Epoch_' + str(num_epochs)+'_Metrics.txt'
        
        model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,num_epochs,fold_num=1)
        fold_total,fold_correct,fold_true,fold_predicted = conf_mat(model_ft,"1")
        fold_accuracies,fold_true_labels,fold_predicted_labels = genel_fold_basarisi(fold_total,fold_correct,fold_true,fold_predicted)
        print("fold 1 analizi bitti")

        dataset_dir = dataset_path + '/fold_2'
        dataloaders, dataset_sizes = prepare_data(dataset_dir)
        model_ft,criterion,optimizer_ft,exp_lr_scheduler=restart_model(opti,learning_rate_selection)
        model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,num_epochs,fold_num=2)
        fold_total,fold_correct,fold_true,fold_predicted = conf_mat(model_ft,"2")
        fold_accuracies,fold_true_labels,fold_predicted_labels = genel_fold_basarisi(fold_total,fold_correct,fold_true,fold_predicted)
        print("fold 2 analizi bitti")

        dataset_dir = dataset_path + '/fold_3'
        dataloaders, dataset_sizes = prepare_data(dataset_dir)
        model_ft,criterion,optimizer_ft,exp_lr_scheduler=restart_model(opti,learning_rate_selection)
        model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,num_epochs,fold_num=3)
        fold_total,fold_correct,fold_true,fold_predicted = conf_mat(model_ft,"3")
        fold_accuracies,fold_true_labels,fold_predicted_labels = genel_fold_basarisi(fold_total,fold_correct,fold_true,fold_predicted)
        print("fold 3 analizi bitti")

        dataset_dir = dataset_path + '/fold_4'
        dataloaders, dataset_sizes = prepare_data(dataset_dir)
        model_ft,criterion,optimizer_ft,exp_lr_scheduler=restart_model(opti,learning_rate_selection)
        model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,num_epochs,fold_num=4)
        fold_total,fold_correct,fold_true,fold_predicted = conf_mat(model_ft,"4")
        fold_accuracies,fold_true_labels,fold_predicted_labels = genel_fold_basarisi(fold_total,fold_correct,fold_true,fold_predicted)
        print("fold 4 analizi bitti")

        dataset_dir = dataset_path + '/fold_5'
        dataloaders, dataset_sizes = prepare_data(dataset_dir)
        model_ft,criterion,optimizer_ft,exp_lr_scheduler=restart_model(opti,learning_rate_selection)
        model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,num_epochs,fold_num=5)
        fold_total,fold_correct,fold_true,fold_predicted = conf_mat(model_ft,"5")
        fold_accuracies,fold_true_labels,fold_predicted_labels = genel_fold_basarisi(fold_total,fold_correct,fold_true,fold_predicted)
        print("fold 5 analizi bitti")

        fold_conf_mat(fold_true_labels,fold_predicted_labels,opti,learning_rate_selection,imageName,txtName,model_ft)
